In [36]:
import os
try:
    os.chdir('Users/giosue.cotugno/mlops_titanic/notebooks/')
except:
    print('already moved')

already moved


# imports

In [37]:

from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep
import os
from azureml.core import Workspace,Experiment, Environment
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# Defining utility function

In [38]:
def getOrCreateCompute(ws:Workspace):
    
    

    aml_compute_target = "testcot"
    try:
        aml_compute = AmlCompute(ws, aml_compute_target)
        print("found existing compute target.")
    except ComputeTargetException:
        print("creating new compute target")

        provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                    min_nodes = 0, 
                                                                    max_nodes = 4)    
        aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
        aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    finally:
        return aml_compute

In [39]:
def createRunConfig(ws):
    
    from azureml.core.runconfig import RunConfiguration
    from azure.ai.ml import MLClient
    from azure.identity import DefaultAzureCredential
    from azureml.core import Environment
    # create a new runconfig object
    run_config = RunConfiguration()
    env = Environment.get(workspace=ws, name='TITANIC', version='1')

    run_config.environment=env

    

    return run_config

# Build dev pipeline

In [40]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')


# Default datastore (Azure blob storage)
def_blob_store = ws.get_default_datastore()
#def_blob_store = Datastore(ws, "workspaceblobstore")
print("Blobstore's name: {}".format(def_blob_store.name))
#Upload file to datastore

# Use a CSV to read in the data set.
file_name = "../data/rawdata/train.csv"

if not os.path.exists(file_name):
    raise Exception(
        'Could not find CSV dataset at "%s". '
        % file_name
    )  # NOQA: E50
# Upload file to default datastore in workspace

mlops-aml-ws
mlops-rg
westeurope
f90533aa-280d-40b9-9949-a7ba0ee9511f
Blobstore's name: workspaceblobstore


# Uploading data to blob storage

In [41]:
target_path = "training-data/"
def_blob_store.upload_files(
    files=[file_name],
    target_path=target_path,
    overwrite=True,
    show_progress=False,
)

blob_input_data = DataReference(
    datastore=def_blob_store,
    data_reference_name="test_data",
    path_on_datastore="training-data/train.csv")


aml_compute = getOrCreateCompute(ws)
run_config = createRunConfig(ws)
#processed_data1 = PipelineData("processed_data1",datastore=def_blob_store)
models_data = PipelineData("models_data",datastore=def_blob_store)

found existing compute target.


## Preprocess step

In [ ]:
source_directory="../src/preprocess/"
preprocess_step = PythonScriptStep(
    script_name="preprocess.py", 
    arguments=["--data", blob_input_data],
    inputs=[blob_input_data],
    compute_target=aml_compute, 
    source_directory=source_directory,
    runconfig=run_config,
    allow_reuse=True
)

In [ ]:
from azureml.pipeline.core import Pipeline
Pipeline(ws, [preprocess_step])

## Training step

In [ ]:
source_directory="../src/train/"
train_step = PythonScriptStep(
    name="Train Model",
    script_name="train.py",
    compute_target=aml_compute,
    source_directory=source_directory,
    outputs=[models_data],
    arguments=[
        "--model",
        models_data
    ],
    runconfig=run_config,
    allow_reuse=True,
)

## Validation step

In [ ]:
source_directory="../src/evaluation/"
evaluate_step = PythonScriptStep(
    name="Evaluate Model ",
    script_name="eval.py",
    compute_target=aml_compute,
    source_directory=source_directory,
    inputs=[models_data],
    arguments=[
        "--model_path",
        models_data,
    ],
    runconfig=run_config,
    allow_reuse=False,
)

# Pipeline creation

In [ ]:
from azureml.pipeline.core import Pipeline
train_step.run_after(preprocess_step)
evaluate_step.run_after(train_step)
steps = [preprocess_step,train_step, evaluate_step]

train_pipeline = Pipeline(workspace=ws, steps=steps)
#train_pipeline._set_experiment_name


In [ ]:
pipeline_run1 = Experiment(ws, 'titanic-pipeline').submit(train_pipeline)

Created step preprocess.py [3c18fb30][2c63d416-1b63-40da-a0f6-5f6e2a4067b1], (This step is eligible to reuse a previous run's output)
Created step Train Model [7944e2c1][f767d24b-7c20-4c2a-9895-d9c8f4ba76be], (This step will run and generate new outputs)
Created step Evaluate Model  [89740306][2d44beba-57db-4028-89c9-fc81bb2c364f], (This step will run and generate new outputs)
Using data reference test_data for StepId [5b95c50c][ee17b8e7-aad7-4670-a604-b0b5a122ca4f], (Consumers of this data are eligible to reuse prior runs.)
Submitted PipelineRun 28f603cf-2193-4102-90fa-87c018d1c077
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/28f603cf-2193-4102-90fa-87c018d1c077?wsid=/subscriptions/f90533aa-280d-40b9-9949-a7ba0ee9511f/resourcegroups/mlops-rg/workspaces/mlops-aml-ws&tid=e99647dc-1b08-454a-bf8c-699181b389ab


In [ ]:
pipeline_run1.wait_for_completion(show_output=True)

PipelineRunId: 28f603cf-2193-4102-90fa-87c018d1c077
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/28f603cf-2193-4102-90fa-87c018d1c077?wsid=/subscriptions/f90533aa-280d-40b9-9949-a7ba0ee9511f/resourcegroups/mlops-rg/workspaces/mlops-aml-ws&tid=e99647dc-1b08-454a-bf8c-699181b389ab
PipelineRun Status: NotStarted
PipelineRun Status: Running


Expected a StepRun object but received <class 'azureml.core.run.Run'> instead.
This usually indicates a package conflict with one of the dependencies of azureml-core or azureml-pipeline-core.
Please check for package conflicts in your python environment


Expected a StepRun object but received <class 'azureml.core.run.Run'> instead.
This usually indicates a package conflict with one of the dependencies of azureml-core or azureml-pipeline-core.
Please check for package conflicts in your python environment


Expected a StepRun object but received <class 'azureml.core.run.Run'> instead.
This usually indicates a package conflict with one of the dependencies of azureml-core or azureml-pipeline-core.
Please check for package conflicts in your python environment






PipelineRun Execution Summary
PipelineRun Status: Failed


ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Pipeline has some failed steps. See child run or execution logs for more details.",
        "messageFormat": "Pipeline has some failed steps. {0}",
        "messageParameters": {},
        "referenceCode": "PipelineHasStepJobFailed",
        "details": []
    },
    "environment": "westeurope",
    "location": "westeurope",
    "time": "2022-12-16T12:16:44.23959Z",
    "componentName": ""
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Pipeline has some failed steps. See child run or execution logs for more details.\",\n        \"messageFormat\": \"Pipeline has some failed steps. {0}\",\n        \"messageParameters\": {},\n        \"referenceCode\": \"PipelineHasStepJobFailed\",\n        \"details\": []\n    },\n    \"envir